# End-to-end NLP: News Headline classifier

### Setup execution role and session

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

### Download News Aggregator Dataset available at the public UCI dataset repository

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

In [ ]:
!unzip NewsAggregatorDataset.zip

In [ ]:
!rm -rf __MACOSX/

In [6]:
#ls

#### Let's visualize the dataset

In [ ]:
import pandas as pd
import mxnet
import re
import numpy as np
import os

In [ ]:
column_names = ["TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
news_dataset = pd.read_csv('newsCorpora.csv', names=column_names, header=None, delimiter='\t')
news_dataset.head()

#### For this exercice we'll only use the title (Headline) of the news story and the category as our target variable

In [ ]:
df=news_dataset[['TITLE',"CATEGORY"]]

In [ ]:
from collections import Counter
Counter(df['CATEGORY'])

The dataset has four categories: Business (b), Science & Technology (t), Entertainment (e) and Health & Medicine (m).

#### Dummy encode the labels

In [ ]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
encoder = preprocessing.LabelEncoder()

docs = df["TITLE"].values

encoder.fit(df["CATEGORY"].values)
encoded_Y = encoder.transform(df["CATEGORY"].values)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

In [ ]:
#bucket = <bucket> # custom bucket name.
s3_bucket = sess.default_bucket()
s3_prefix = 'news'

In [ ]:
list(encoder.classes_)

In [ ]:
encoded_Y

#### Tokenize documents and set fixed sequence lengths for input feature dimension.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(vocab_size)
# pad documents to a max length of 4 words
max_length = 40
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

In [ ]:
docs[0]

### Import word embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip && unzip glove.6B.zip

In [ ]:
rm 2pageSessions.csv glove.6B.200d.txt glove.6B.50d.txt glove.6B.300d.txt glove.6B.zip

##### Create embedding matrix

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./vectors.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [118]:
#embeddings_index

In [23]:
#print(t.word_index)

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
mkdir ./data/ ./data/embeddings/

In [ ]:
#embedding_matrix.dump("ingredients-embedding-matrix.dat")
np.save(file="./data/embeddings/docs-embedding-matrix",
        arr=embedding_matrix,
        allow_pickle=False)
print(embedding_matrix.shape)

### Train, test split

In this section we will prep the data for ingestion for the algortihm. Split the data set in train and test samples and uplad the data to S3

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_docs, dummy_y, test_size=0.2, random_state=42)

In [ ]:
!mkdir data/train/ data/test/

In [ ]:
np.save('./data/train/train_X.npy', X_train)
np.save('./data/train/train_Y.npy', y_train)
np.save('./data/test/test_X.npy', X_test)
np.save('./data/test/test_Y.npy', y_test)

In [ ]:
traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)
embeddings_s3_prefix='{}/data/embeddings'.format(s3_prefix)
output_s3 = 's3://{}/{}/models/'.format(s3_bucket, s3_prefix)
code_location_s3 = 's3://{}/{}/codes'.format(s3_bucket, s3_prefix)

In [ ]:
train_s3 = sess.upload_data(path='./data/train/', bucket=s3_bucket, key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', bucket=s3_bucket, key_prefix=testdata_s3_prefix)
embeddings_s3 = sess.upload_data(path='./data/embeddings/', bucket=s3_bucket, key_prefix=embeddings_s3_prefix)


In [ ]:
inputs = {'train':train_s3, 'test': test_s3, 'embeddings': embeddings_s3}

print(inputs)

In [ ]:
import sagemaker
from sagemaker.mxnet import MXNet

### Define hyperparameters to push to algorithm

In [ ]:
hyperparameters = {'epochs': 5, 'vocab_size':vocab_size, 'num_classes':encoder.classes_.size}

In [ ]:
mxnet_estimator = MXNet(entry_point='keras_script_mxnet.py',
                       source_dir='./tf-src',
                        role=role,
                        train_instance_type='ml.m4.xlarge',
                        train_instance_count=1,
                        framework_version='1.3.0',
                        py_version='py3',
                        hyperparameters=hyperparameters)
mxnet_estimator.fit(inputs)

In [ ]:
import boto3
s3 = boto3.resource('s3')

key = mxnet_estimator.model_data[mxnet_estimator.model_data.find("/", 5)+1:]
s3.Bucket(s3_bucket).download_file(key, 'model.tar.gz')

In [ ]:
model_path='model.tar.gz'
from sagemaker.mxnet import MXNet, MXNetModel

sagemaker_model = MXNetModel(model_data = model_path,
                             role = role,
                             entry_point = 'default_classifier.py',
                             py_version='py3')

In [ ]:
predictor = mxnet_estimator.deploy(initial_instance_count=1,
                           instance_type='ml.t2.medium')

In [ ]:
display(predictor.accept, predictor.content_type, predictor.deserializer, predictor.endpoint, predictor.sagemaker_session, predictor.serializer)

In [38]:
#predictor.predict(padded_example.tolist())

In [ ]:
import boto3
s3 = boto3.resource('s3')

key = mxnet_estimator.model_data[mxnet_estimator.model_data.find("/", 5)+1:]
s3.Bucket(s3_bucket).download_file(key, 'model.tar.gz')

In [ ]:
!tar -xvzf model.tar.gz

In [ ]:
from keras.models import load_model
loaded_model = load_model("model.hd5")

In [ ]:
example_doc=['Senate prepares to vote on dueling plans to end shutdown']
# integer encode the document
encoded_example = t.texts_to_sequences(example_doc)

# pad documents to a max length of 4 words
max_length = 40
padded_example = pad_sequences(encoded_example, maxlen=max_length, padding='post')

In [ ]:
loaded_model.predict(padded_example)